In [16]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

In [34]:
# All code referenced from Milestone2.ipynb file provided by Gittu in DSCI 525

In [17]:
# Necessary metadata
article_id = 14226968  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "/srv/data/my_shared_data_folder/"

In [18]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'allyears.csv.zip',
  'supplied_md5': '9e046ac05ecd2c32a256a47dd1098b81',
  'computed_md5': '9e046ac05ecd2c32a256a47dd1098b81',
  'id': 26844650,
  'download_url': 'https://ndownloader.figshare.com/files/26844650',
  'size': 2405908113},
 {'is_link_only': False,
  'name': 'individual_years.zip',
  'supplied_md5': '921da748974b07b2a70bbfcc04535a77',
  'computed_md5': '921da748974b07b2a70bbfcc04535a77',
  'id': 26863682,
  'download_url': 'https://ndownloader.figshare.com/files/26863682',
  'size': 1896206676},
 {'is_link_only': False,
  'name': 'combined_model_data.csv.zip',
  'supplied_md5': '7638434c44a7d29cbb29fe200b4fd65d',
  'computed_md5': '7638434c44a7d29cbb29fe200b4fd65d',
  'id': 27515426,
  'download_url': 'https://ndownloader.figshare.com/files/27515426',
  'size': 821308997},
 {'is_link_only': False,
  'name': 'combined_model_data_parti.parquet.zip',
  'supplied_md5': '02f4e3df8d16580a02291de225072689',
  'computed_md5': '02f4e3df8d16580a02

In [19]:
files_to_dl = ["combined_model_data_parti.parquet.zip"]  ## Please download the partitioned 
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [20]:
with zipfile.ZipFile(os.path.join(output_directory, "combined_model_data_parti.parquet.zip"), 'r') as f:
    f.extractall(output_directory)

In [21]:
# Code referenced from Milestone2.ipynb file provided by Gittu in DSCI 525
## IMPORTANT: make sure you dont include you secret and key when submitting the notebook
aws_credentials ={"key": "","secret": ""} ## dont include you secret and key when submitting the notebook
df = pd.read_parquet("s3://mds-s3-student96/combined_model_data_parti.parquet", storage_options=aws_credentials)

In [22]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


In [23]:
# shape should be (62513863, 7)
df.shape

(62513863, 7)

In [24]:
syd_lat = -33.86
syd_lon = 151.21

syd_df = df.query("((lat_min < @syd_lat) & (lat_max > @syd_lat)) & ((lon_min < @syd_lon) & (lon_max > @syd_lon))")
syd_df = syd_df.drop(columns=["lat_min", "lat_max", "lon_min", "lon_max"])
syd_df.index = syd_df["time"]
syd_df = syd_df.drop(columns = "time")

syd_df.head()

,rain (mm/day),model
time,,
1889-01-01 12:00:00,0.040427,ACCESS-CM2
1889-01-02 12:00:00,0.073777,ACCESS-CM2
1889-01-03 12:00:00,0.232656,ACCESS-CM2
1889-01-04 12:00:00,0.911319,ACCESS-CM2
1889-01-05 12:00:00,0.698013,ACCESS-CM2


In [25]:
# shape should be (1150049, 2)
syd_df.shape

(1150049, 2)

In [26]:
syd_df = syd_df.pivot(columns = "model", values = "rain (mm/day)")
syd_df.head()

model,ACCESS-CM2,ACCESS-ESM1-5,AWI-ESM-1-1-LR,BCC-CSM2-MR,BCC-ESM1,CMCC-CM2-HR4,CMCC-CM2-SR5,CMCC-ESM2,CanESM5,EC-Earth3-Veg-LR,...,MIROC6,MPI-ESM-1-2-HAM,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0,NESM3,NorESM2-LM,NorESM2-MM,SAM0-UNICON,TaiESM1
time,,,,,,,,,,,,,,,,,,,,,
1889-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1889-01-01 12:00:00,0.040427,1.814552,35.579336,4.268112,0.001107,11.410537,3.322009e-08,2.668800,1.321215,1.515293,...,0.648031,4.244226e-13,1.390174e-13,6.537884e-05,3.445495e-06,15.760960,0.000048,2.451075,0.221324,2.257933
1889-01-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1889-01-02 12:00:00,0.073777,0.303965,4.596520,1.190141,0.000102,4.014984,1.312700e+00,0.946211,2.788724,4.771375,...,1.099883,4.409552e+00,1.222283e-01,1.049131e-13,4.791993e-09,0.367551,0.435086,0.477231,3.757179,2.287381
1889-01-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
syd_df = syd_df.resample("D").mean()
syd_df.head()

model,ACCESS-CM2,ACCESS-ESM1-5,AWI-ESM-1-1-LR,BCC-CSM2-MR,BCC-ESM1,CMCC-CM2-HR4,CMCC-CM2-SR5,CMCC-ESM2,CanESM5,EC-Earth3-Veg-LR,...,MIROC6,MPI-ESM-1-2-HAM,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0,NESM3,NorESM2-LM,NorESM2-MM,SAM0-UNICON,TaiESM1
time,,,,,,,,,,,,,,,,,,,,,
1889-01-01,0.040427,1.814552,35.579336,4.268112e+00,1.107466e-03,11.410537,3.322009e-08,2.668800,1.321215,1.515293,...,0.648031,4.244226e-13,1.390174e-13,6.537884e-05,3.445495e-06,1.576096e+01,4.759651e-05,2.451075,0.221324,2.257933
1889-01-02,0.073777,0.303965,4.596520,1.190141e+00,1.015323e-04,4.014984,1.312700e+00,0.946211,2.788724,4.771375,...,1.099883,4.409552e+00,1.222283e-01,1.049131e-13,4.791993e-09,3.675510e-01,4.350863e-01,0.477231,3.757179,2.287381
1889-01-03,0.232656,0.019976,5.927467,1.003845e-09,1.760345e-05,9.660565,9.103720e+00,0.431999,0.003672,4.233980,...,3.931751,2.269300e-01,3.762301e-01,9.758706e-14,6.912302e-01,1.562869e-01,9.561101e+00,0.023083,0.253357,1.199909
1889-01-04,0.911319,13.623777,8.029624,8.225225e-02,1.808932e-01,3.951528,1.317160e+01,0.368693,0.013578,15.252495,...,0.627365,2.344586e-02,4.214019e-01,7.060915e-03,3.835721e-02,2.472226e-07,5.301038e-01,0.002699,2.185454,2.106737
1889-01-05,0.698013,0.021048,2.132686,2.496841e+00,4.708019e-09,2.766362,1.822940e+01,0.339267,0.002468,11.920356,...,0.019777,4.270161e-13,1.879692e-01,4.504985e+00,3.506923e-07,1.949792e-13,1.460928e-10,0.001026,2.766507,1.763335


In [28]:
# Code referenced from Milestone2.ipynb file provided by Gittu in DSCI 525
# Read in observed_daily_rainfall_SYD.csv
aws_credentials ={"key": "","secret": ""} ## dont include you secret and key when submitting the notebook
df_obs = pd.read_csv("s3://mds-s3-student96/observed_daily_rainfall_SYD.csv", storage_options=aws_credentials, 
                 parse_dates = True, index_col = 0)

In [29]:
df_obs.head()

,rain (mm/day)
time,
1889-01-01,0.006612
1889-01-02,0.090422
1889-01-03,1.401452
1889-01-04,14.869798
1889-01-05,0.467628


In [30]:
df_obs.columns = ["Observed"]

In [31]:
# Combine Observed column with the Sydney data
combined_df = pd.concat([syd_df, df_obs], axis=1)
combined_df.head()

,ACCESS-CM2,ACCESS-ESM1-5,AWI-ESM-1-1-LR,BCC-CSM2-MR,BCC-ESM1,CMCC-CM2-HR4,CMCC-CM2-SR5,CMCC-ESM2,CanESM5,EC-Earth3-Veg-LR,...,MPI-ESM-1-2-HAM,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0,NESM3,NorESM2-LM,NorESM2-MM,SAM0-UNICON,TaiESM1,Observed
time,,,,,,,,,,,,,,,,,,,,,
1889-01-01,0.040427,1.814552,35.579336,4.268112e+00,1.107466e-03,11.410537,3.322009e-08,2.668800,1.321215,1.515293,...,4.244226e-13,1.390174e-13,6.537884e-05,3.445495e-06,1.576096e+01,4.759651e-05,2.451075,0.221324,2.257933,0.006612
1889-01-02,0.073777,0.303965,4.596520,1.190141e+00,1.015323e-04,4.014984,1.312700e+00,0.946211,2.788724,4.771375,...,4.409552e+00,1.222283e-01,1.049131e-13,4.791993e-09,3.675510e-01,4.350863e-01,0.477231,3.757179,2.287381,0.090422
1889-01-03,0.232656,0.019976,5.927467,1.003845e-09,1.760345e-05,9.660565,9.103720e+00,0.431999,0.003672,4.233980,...,2.269300e-01,3.762301e-01,9.758706e-14,6.912302e-01,1.562869e-01,9.561101e+00,0.023083,0.253357,1.199909,1.401452
1889-01-04,0.911319,13.623777,8.029624,8.225225e-02,1.808932e-01,3.951528,1.317160e+01,0.368693,0.013578,15.252495,...,2.344586e-02,4.214019e-01,7.060915e-03,3.835721e-02,2.472226e-07,5.301038e-01,0.002699,2.185454,2.106737,14.869798
1889-01-05,0.698013,0.021048,2.132686,2.496841e+00,4.708019e-09,2.766362,1.822940e+01,0.339267,0.002468,11.920356,...,4.270161e-13,1.879692e-01,4.504985e+00,3.506923e-07,1.949792e-13,1.460928e-10,0.001026,2.766507,1.763335,0.467628


In [32]:
# shape should be (46020,26)
combined_df.shape

(46020, 26)

In [33]:
# Save the combined csv file to the S3 bucket
combined_df.to_csv("s3://mds-s3-student49/output/ml_data_SYD.csv", storage_options=aws_credentials)